In [1]:
#!/usr/bin/env python
import os
import sys
import rospy
import copy
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from math import *
from numpy.linalg import inv
import scipy.io
from mat4py import loadmat
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from scoop.msg import triangle_pose
from sensor_msgs.msg import Image, CameraInfo
import message_filters
from visualization_msgs.msg import Marker
from visualization_msgs.msg import MarkerArray
from geometry_msgs.msg import Point
import random
import urx
import math3d as m3d
import logging
import matplotlib.pyplot as plt
from geometry_msgs.msg import WrenchStamped
import time
rospy.init_node('scoop', anonymous=True) #initialize the node



In [2]:
rospy.logwarn("Client test: Starting sending goals")
rob = urx.Robot("192.168.1.102")
logging.basicConfig(level=logging.WARN)
rob.set_tcp((0, 0, 0, 0, 0, 0))

[WARN] [1612520071.003172]: Client test: Starting sending goals


In [3]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()

True

In [4]:
import serial
import signal
import time

serialPort = "/dev/Arduino"
baudRate = 57600
ser = serial.Serial(serialPort, baudRate, timeout=0.5)
def myHandler(signum, frame):
    pass

In [5]:
'''
def go_to_home():
    
    Hong_joint0 = radians(101.92)
    Hong_joint1 = radians(-82.25)
    Hong_joint2 = radians(-128.90)
    Hong_joint3 = radians(-58.88)
    Hong_joint4 = radians(89.35)
    Hong_joint5 = radians(-78.02)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
'''


def go_to_home():
    
    Hong_joint0 = radians(101.94)
    Hong_joint1 = radians(-83.72)
    Hong_joint2 = radians(-130.33)
    Hong_joint3 = radians(-55.98)
    Hong_joint4 = radians(89.33)
    Hong_joint5 = radians(-78.02)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)

    
def go_to_safe_place():
    
    Hong_joint0 = radians(101.74)
    Hong_joint1 = radians(-76.84)
    Hong_joint2 = radians(-120.83)
    Hong_joint3 = radians(-72.33)
    Hong_joint4 = radians(89.40)
    Hong_joint5 = radians(-78.12)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_end_place():
    
    Hong_joint0 = radians(106.13)
    Hong_joint1 = radians(-60.48)
    Hong_joint2 = radians(-130.69)
    Hong_joint3 = radians(-78.86)
    Hong_joint4 = radians(89.68)
    Hong_joint5 = radians(-73.78)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)

In [6]:
def stone_position_in_the_world(pose_x, pose_y, pose_z):
    if pose_z==0.0:
        return [None, None, None]
    while True:
        rob.set_tcp((0, 0.0, 0, 0, 0, 0))    
        camPstone = np.array([pose_x, pose_y, pose_z, 1])
        eeTcam = m3d.Transform()
        eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
        eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
        eeTcam.orient.rotate_xb(eeTcam_e[0])
        eeTcam.orient.rotate_yb(eeTcam_e[1])
        eeTcam.orient.rotate_zb(eeTcam_e[2])
        tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
        eeTstone = np.matmul(eeTcam.get_matrix(), camPstone)
        tcpTstone = np.matmul(tcpTee, np.transpose(eeTstone))
        worldTstone = np.matmul(rob.get_pose().get_matrix(), tcpTstone)
        worldTstone = worldTstone.squeeze().tolist()[0][0:-1]
        if worldTstone[2]<0:
            continue
        else:
            return [worldTstone[0]+0.02+0.0025+0.003, worldTstone[1]+0.02148-0.00353-0.0006, max(worldTstone[2], 0.035)]

In [7]:
import json
def hori_stone_pose_callback(msg):
    global hori_pose_Go_stone_set, hori_Go_stone_number, yaw_pitch_normal_set, hori_box_set, hori_box_pixel_set, hori_json_error
    hori_Go_stone_number = int(msg.hori_Go_stone_number)
    if hori_Go_stone_number == 0:
        hori_pose_Go_stone_set = []
        yaw_pitch_normal_set = []
        hori_box_set = []
        hori_box_pixel_set = []
    else:
        #try:
        if True:
            pose_x = msg.x
            pose_y = msg.y
            pose_z = msg.z
            yaw = msg.yaw
            pitch = msg.pitch
            normal = msg.normal
            hori_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
            yaw_pitch_normal_set.append([yaw,pitch,normal])
            box = json.loads(msg.box)
            box = [stone_position_in_the_world(box[i][0], box[i][1], pose_z)[0:2] for i in range(len(box))]
            hori_box_set.append(box)
            box_pixel = json.loads(msg.box_pixel)
            box_pixel = [[box_pixel[i][0], box_pixel[i][1]] for i in range(len(box_pixel))]
            hori_box_pixel_set.append(box_pixel)
        #except:
            #hori_json_error = True
hori_pose_topic = '/triangle_pose' #"/usb_cam/image_raw"
rospy.Subscriber(hori_pose_topic, triangle_pose, hori_stone_pose_callback)

In [8]:
is_send_stone_img = 0
stone_img_count = 0
bridge = CvBridge()

def image_callback(color, a_depth):
    global is_send_stone_img, stone_img_count, cv2_img, cv2_depth_img, depth_array
    if is_send_stone_img == 1:
        print "dsag"
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        rospy.sleep(0.5)
        cv2_img = bridge.imgmsg_to_cv2(color, "bgr8")
        cv2.imwrite('/home/terry/Mask_RCNN/samples/triangle/JPEGImages/'+str(stone_img_count)+'.jpeg', cv2_img)
        cv2_depth_img = bridge.imgmsg_to_cv2(a_depth, desired_encoding="passthrough")
        depth_array = np.array(cv2_depth_img, dtype=np.float32)
        cv2.imwrite('/home/terry/Mask_RCNN/samples/triangle/depth/'+str(stone_img_count)+'.jpeg', cv2_depth_img)
        np.save('/home/terry/Mask_RCNN/samples/triangle/depth/'+str(stone_img_count)+'.npy', depth_array)
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        stone_img_count = stone_img_count + 1
        is_send_stone_img = 0

In [9]:
image_color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
image_aligned_depth_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
ts = message_filters.TimeSynchronizer([image_color_sub, image_aligned_depth_sub], 10)
ts.registerCallback(image_callback)

0

In [10]:
def finger_tip_position_wrt_gripper_frame(alpha, l0, l1, l2l, l2r, flength_l, flength_r):
    alpha=alpha*pi/180
    left_fingertip_position=[-l0-l1*sin(alpha)+l2l, -l1*cos(alpha)-flength_l]
    right_fingertip_position=[l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r]
    return left_fingertip_position, right_fingertip_position

def pos_flength_adjust_finger_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_r):
    theta=theta*pi/180
    current_flength_l = flength_r - (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)  
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r) 
    traslating_wst_g = [current_right_fingertip_pos_g[0]-next_right_fingertip_pos_g[0], current_right_fingertip_pos_g[1]-next_right_fingertip_pos_g[1]]
   # print np.array([[traslating_wst_g[0]], [traslating_wst_g[1]]])
    traslating_wst_world = np.array([[sin(theta), -cos(theta)], [cos(theta), sin(theta)]]).dot(np.array([[traslating_wst_g[0]], [traslating_wst_g[1]]])).ravel().tolist()
    next_left_fingertip_pos_g = [next_left_fingertip_pos_g[0]+traslating_wst_g[0], next_left_fingertip_pos_g[1]+traslating_wst_g[1]]
    next_right_fingertip_pos_g = [next_right_fingertip_pos_g[0]+traslating_wst_g[0], next_right_fingertip_pos_g[1]+traslating_wst_g[1]]
    extension_distance = -(next_right_fingertip_pos_g[1]+(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r)/tan(theta)-next_left_fingertip_pos_g[1])
    return traslating_wst_world, extension_distance

def pos_flength_adjust_thumb_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_r):
    theta=theta*pi/180
    current_flength_l = flength_r - (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)  
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r) 
    traslating_wst_g = [current_right_fingertip_pos_g[0]-next_right_fingertip_pos_g[0], current_right_fingertip_pos_g[1]-next_right_fingertip_pos_g[1]]
    traslating_wst_world = np.array([[sin(theta), -cos(theta)], [cos(theta), sin(theta)]]).dot(np.array([[traslating_wst_g[0]], [traslating_wst_g[1]]])).ravel().tolist()
    next_left_fingertip_pos_g = [next_left_fingertip_pos_g[0]+traslating_wst_g[0], next_left_fingertip_pos_g[1]+traslating_wst_g[1]]
    next_right_fingertip_pos_g = [next_right_fingertip_pos_g[0]+traslating_wst_g[0], next_right_fingertip_pos_g[1]+traslating_wst_g[1]]
    extension_distance = -(next_right_fingertip_pos_g[1]+(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r)/tan(theta)-next_left_fingertip_pos_g[1])
    traslating_wst_world[0]=traslating_wst_world[0]-((2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)-(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r))/sin(theta)
    return traslating_wst_world, extension_distance

In [11]:
from __future__ import division
import pybullet as p
import pybullet_data
import numpy as np
import time
import argparse
import pybullet_utils
from collision_utils import get_collision_fn
from scipy.spatial.transform import Rotation as R
from math import *

THUMB_JOINT_INDICES = [0]

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--birrt', action='store_true', default=False)
    parser.add_argument('--smoothing', action='store_true', default=False)
    args = parser.parse_args(args=[])
    return args

def set_joint_positions(body, joints, values):
    assert len(joints) == len(values)
    for joint, value in zip(joints, values):
        p.resetJointState(body, joint, value)
args = get_args()
physicsClient = p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setGravity(0, 0, -9.8)
p.configureDebugVisualizer(p.COV_ENABLE_GUI, False)
p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS, True)
p.resetDebugVisualizerCamera(cameraDistance=1.400, cameraYaw=58.000, cameraPitch=-42.200, cameraTargetPosition=(0.0, 0.0, 0.0))
CLIENT = 0
# load objects
finger = p.loadURDF('/home/terry/catkin_ws/src/scoop/models/urdf/Finger-2.9-urdf.urdf', basePosition=[0, 0, 0], baseOrientation=[0,0,1,0], useFixedBase=False)
thumb = p.loadURDF('/home/terry/catkin_ws/src/scoop/models/urdf/Assem-1.9-urdf.urdf', basePosition=[0, 0, 0], baseOrientation=[0,0,1,0],useFixedBase=False)
bowl_circular = p.loadURDF('/home/terry/catkin_ws/src/scoop/models/urdf/Bowl-circular-urdf.urdf', basePosition=[0.05, 0.69, 0.03], baseOrientation=[0,0,0,1], useFixedBase=True)
obstacles = [bowl_circular]

def skim_grasp_collision_check(Go_stone_pose, rotate_angle0, rotate_angle1, rotate_angle2, psi=30*pi/180): # radius
    #time0=time.time()
    start_conf_thumb = [-0.0167]
    fingerbaseOrientation_raw = R.from_dcm([[0, 1, 0], [1, 0, 0], [0, 0, -1]]) * R.from_euler('z', rotate_angle0, degrees=False) * R.from_euler('x', rotate_angle1, degrees=False) * R.from_euler('y', rotate_angle2, degrees=False)
    fingerbaseOrientation = fingerbaseOrientation_raw.as_quat().tolist()
    ThumbTFinger_normal = fingerbaseOrientation_raw.as_dcm()[:,0].tolist()
    fingerbasePosition = [Go_stone_pose[0]+0.006*ThumbTFinger_normal[0], Go_stone_pose[1]+0.006*ThumbTFinger_normal[1], Go_stone_pose[2]+0.006*ThumbTFinger_normal[2]]
    p.resetBasePositionAndOrientation(finger, fingerbasePosition, fingerbaseOrientation)
    finger_base_position = list(p.getLinkState(finger, 0)[0])
    #print finger_base_position
    thumbbaseOrientation = fingerbaseOrientation
    thumbbasePosition = [finger_base_position[0]-0.043*ThumbTFinger_normal[0], finger_base_position[1]-0.043*ThumbTFinger_normal[1], finger_base_position[2]-0.043*ThumbTFinger_normal[2]] 
    #print thumbbasePosition
    set_joint_positions(thumb, THUMB_JOINT_INDICES, start_conf_thumb)
    p.resetBasePositionAndOrientation(thumb, thumbbasePosition, thumbbaseOrientation)
    #collision_fn0 = len(p.getClosestPoints(bodyA=finger, bodyB=bowl_circular, distance=0, physicsClientId=CLIENT)) != 0
    collision_fn1 = len(p.getClosestPoints(bodyA=thumb, bodyB=bowl_circular, distance=0, physicsClientId=CLIENT)) != 0
    fingerbaseOrientation = (fingerbaseOrientation_raw * R.from_euler('y', psi, degrees=False)).as_quat().tolist()
    p.resetBasePositionAndOrientation(finger, fingerbasePosition, fingerbaseOrientation)
    finger_base_position = list(p.getLinkState(finger, 0)[0])
    #print fingerbaseOrientation
    thumbbaseOrientation = fingerbaseOrientation
    ThumbTFinger_normal = R.from_quat(fingerbaseOrientation).as_dcm()[:,0].tolist()
    thumbbasePosition = [finger_base_position[0]-0.043*ThumbTFinger_normal[0], finger_base_position[1]-0.043*ThumbTFinger_normal[1], finger_base_position[2]-0.043*ThumbTFinger_normal[2]]  
    p.resetBasePositionAndOrientation(thumb, thumbbasePosition, thumbbaseOrientation)
    #collision_fn2 = len(p.getClosestPoints(bodyA=finger, bodyB=bowl_circular, distance=0, physicsClientId=CLIENT)) != 0
    collision_fn3 = len(p.getClosestPoints(bodyA=thumb, bodyB=bowl_circular, distance=0, physicsClientId=CLIENT)) != 0
    #print collision_fn0, collision_fn1, collision_fn2, collision_fn3
    result = collision_fn1 or collision_fn3
    return result

#kim_grasp_collision_check([0.0868192929930581, 0.68349850703534, 0.037], -1.43239448358, 0.00962355024892, -0.00444143180659, 0)

In [20]:
def skim_grasp_execute(Go_index):
    print 'Go_index', index
    global hori_pose_Go_stone_set, yaw_pitch_normal_set, hori_box_set, hori_box_pixel_set, depth_array
    pose_x = hori_pose_Go_stone_set[Go_index][0]
    pose_y = hori_pose_Go_stone_set[Go_index][1]
    pose_z = hori_pose_Go_stone_set[Go_index][2]
    normal=yaw_pitch_normal_set[Go_index][2]
    ####################### change tcp pose ############################
    rob.set_tcp((0.01475, 0, 0.343, 0, 0, 0))  
    rospy.sleep(0.5)
    ####################################################################
    go_to_safe_place()    
    worldPstone = np.array([pose_x, pose_y, pose_z, 1])
    #print 'worldPstone', worldPstone
    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))
    #print "tcpTstone", tcpTstone
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    #print eeTcam
    
    yaxis = (0, 1, 0)
    zaxis = (0, 0, 1)
    Ry = tf.transformations.rotation_matrix(pi/2, yaxis)
    Rz = tf.transformations.rotation_matrix(-pi/2, zaxis)
    eeTtcp = np.matmul(Ry, Rz)
    ####################### change tcp pose ############################
    eeTtcp[:3,3] = np.array([0.3430,-0.01475,0])
    ####################################################################
    tcpTee = inv(eeTtcp)
    #print tcpTee

    n_normalize = normal / np.linalg.norm(normal)
    #print "n_normalize", n_normalize
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    #print n_normalize
    eeTnormal = np.matmul(np.array([[0, 0, 1, 0],[0, 1, 0, 0], [-1,0,0,0], [0,0,0,1]]), n_normalize)
    #print eeTnormal
    tcpTnormal = np.matmul(tcpTee[0:3,0:3], eeTnormal[0:3])
    #print tcpTee
    #print "tcpTnormal", tcpTnormal
    #tcpTnormal = [0,0,-1]
    worldTnormal = [tcpTnormal[1], tcpTnormal[0], tcpTnormal[2]]
    print 'worldTnormal', worldTnormal
    box = hori_box_set[Go_index]
    box_pixel = hori_box_pixel_set[Go_index]
    
    vertex0 = box[0]
    vertex0_pixel = box_pixel[0]
    vertex0_opposite = [(box[1][0]+box[2][0])/2, (box[1][1]+box[2][1])/2]
    vertex0_opposite_pixel = [(box_pixel[1][0]+box_pixel[2][0])/2, (box_pixel[1][1]+box_pixel[2][1])/2]
    vertex1 = box[1]
    vertex1_pixel = box_pixel[1]
    vertex1_opposite = [(box[0][0]+box[2][0])/2, (box[0][1]+box[2][1])/2]
    vertex1_opposite_pixel = [(box_pixel[0][0]+box_pixel[2][0])/2, (box_pixel[0][1]+box_pixel[2][1])/2]
    vertex2 = box[2]
    vertex2_pixel = box_pixel[2]
    vertex2_opposite = [(box[0][0]+box[1][0])/2, (box[0][1]+box[1][1])/2]
    vertex2_opposite_pixel = [(box_pixel[0][0]+box_pixel[1][0])/2, (box_pixel[0][1]+box_pixel[1][1])/2]
    center = hori_pose_Go_stone_set[Go_index]
    dir0 = [vertex0[0]-center[0], vertex0[1]-center[1], 0]
    dir0 = (np.array(dir0)/np.linalg.norm(np.array(dir0))).tolist()
    dir1 = [vertex1[0]-center[0], vertex1[1]-center[1], 0] 
    dir1 = (np.array(dir1)/np.linalg.norm(np.array(dir1))).tolist()
    dir2 = [vertex2[0]-center[0], vertex2[1]-center[1], 0] 
    dir2 = (np.array(dir2)/np.linalg.norm(np.array(dir2))).tolist()
    high_to_low_dir_set = []
    if abs(np.dot(dir0, worldTnormal))<0.1:
        high_to_low_dir_set = [dir0, dir1, dir2]
    else:
        if np.dot(dir0, worldTnormal)>0:
            high_to_low_dir_set.append(dir0)
        if np.dot(dir1, worldTnormal)>0:
            high_to_low_dir_set.append(dir1)
        if np.dot(dir2, worldTnormal)>0:
            high_to_low_dir_set.append(dir2)
    for high_to_low_dir in high_to_low_dir_set:
        rotate_angle = atan2(high_to_low_dir[1], high_to_low_dir[0])-atan2(1,0)
        if rotate_angle > 0:
            rotate_angle = rotate_angle % (2*pi)
        else:
            rotate_angle = rotate_angle % (-2*pi)
        rotate_angle0 = -rotate_angle
        rotate_angle0_matrix = np.array([[cos(rotate_angle0), -sin(rotate_angle0), 0], [sin(rotate_angle0), cos(rotate_angle0), 0], [0,0,1]])
        tcp2normal_after_rotation0 = np.matmul(np.linalg.inv(rotate_angle0_matrix), tcpTnormal[0:3])
        rotate_angle1 = atan(-tcp2normal_after_rotation0[1]/tcp2normal_after_rotation0[2])
        rotate_angle2 = asin(-tcp2normal_after_rotation0[0])    
        if abs(worldTnormal[2])>0.9:
            rotate_angle1 = 0
            rotate_angle2 = 0
        if str(rotate_angle0) == 'nan' or str(rotate_angle1) == 'nan' or str(rotate_angle2) == 'nan':
            continue
        if abs(rotate_angle1)>(35*pi/180.0) or abs(rotate_angle2)>(35*pi/180.0):
            continue
        if abs(rotate_angle1)>(10*pi/180.0):
            psi = 40*pi/180
        else:
            psi = 30*pi/180
        if skim_grasp_collision_check(hori_pose_Go_stone_set[Go_index], rotate_angle0, rotate_angle1, rotate_angle2, psi)==True:
            print 'collision'
            continue
        Robotiq.goto(robotiq_client, pos=3*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1060')
        signal.setitimer(signal.ITIMER_REAL, 0)
        rob.movel_tool((tcpTstone[0], tcpTstone[1]-0.001, 0, 0, 0, 0), acc=0.8, vel=2, wait=True)
        print tcpTstone[2]
        rob.movel_tool((0,0,0,0,0,rotate_angle0), acc=0.5, vel=0.8, wait=True)
        rob.translate_tool((0, 0, tcpTstone[2]-0.015), acc=0.8, vel=1.5)
        rob.movel_tool((0,0,0,rotate_angle1,0,0), acc=0.5, vel=0.8, wait=True)
        rob.movel_tool((0,0,0,0,rotate_angle2,0), acc=0.5, vel=0.8, wait=True)
        ###########################Choose d and psi######################################
        if rotate_angle0>0:
            rob.translate_tool((0.010, 0, 0), acc=0.06, vel=0.08)
        else:
            rob.translate_tool((0.016, 0, 0), acc=0.06, vel=0.08)
        rob.movel_tool((0,0,0,0,psi,0), acc=0.3, vel=0.8, wait=True)
        #################################################################################
        rob.translate((0, 0, -0.02), acc=0.01, vel=0.05, wait=False)
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        ini_wrench = wrench.wrench.torque.y
        time0 = time.time()
        num_large_force = 0
        while True:
            if num_large_force == 3:
                rob.stopl()
                break
            wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
            if pose_z<0.04:
                if wrench.wrench.torque.y>ini_wrench+0.3:   
                    num_large_force += 1
            else:
                if wrench.wrench.torque.y>ini_wrench+0.1:   
                    num_large_force += 1
            if time.time()-time0>2.8:  
                break
        time.sleep(0.1)
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        ini_wrench = wrench.wrench.torque.y
        for current_finger_length_extend in np.arange(0,80,4):
            current_finger_length = 60+current_finger_length_extend #Change
            signal.setitimer(signal.ITIMER_REAL, 0.001)
            ser.write('1'+str(current_finger_length))
            signal.setitimer(signal.ITIMER_REAL, 0)
            time.sleep(0.2)        
            current_fingerlength_int=current_finger_length
            wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
            if wrench.wrench.torque.y>ini_wrench+0.5:
                break
        print current_fingerlength_int  
        angle_step=0.2
        for angle in np.arange(3,-2,-angle_step):
            translate_dir_dis, extension_distance = pos_flength_adjust_finger_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.019, 0.01, 0.125)
            old_fingerlength_int=current_fingerlength_int
            current_fingerlength_int=round(max(current_fingerlength_int+extension_distance*(180.0/0.03),0))
            signal.signal(signal.SIGALRM, myHandler)
            signal.setitimer(signal.ITIMER_REAL, 0.005)
            ser.write('1'+str(current_fingerlength_int))
            signal.setitimer(signal.ITIMER_REAL, 0)   
            if abs(current_fingerlength_int-old_fingerlength_int)!=0 and current_fingerlength_int<165:
                Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
                rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
            else:
                time.sleep(0.01)
        Robotiq.goto(robotiq_client, pos=-2*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
        rospy.sleep(0.5)
        rob.translate_tool((0, 0, -0.12), acc=0.3, vel=0.8, wait=True)
        go_to_end_place()
        rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
        Robotiq.goto(robotiq_client, pos=2*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)
        rospy.sleep(1)
        ser.write('1000')
        rospy.sleep(1)
        rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True) 
        return True
    else:
        return False

In [21]:
def find_alpha_from_distance(distance, l0, l1, l2l, l2r):
    alpha = asin((distance+l2l+l2r-2*l0)/2/l1)*180/pi
    return alpha

def direct_pick_execute(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=0.03, speed=0.1, force=10, block=True)
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1155')
    signal.setitimer(signal.ITIMER_REAL, 0)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)
    pos_x = rob.get_pose().pos.x
    pos_y = rob.get_pose().pos.y
    pos_z = rob.get_pose().pos.z
    rospy.sleep(0.5)
    rob.translate((ver_pose_Go_stone_set[Go_index][0]-pos_x-0.003,ver_pose_Go_stone_set[Go_index][1]-pos_y,0), acc=0.8, vel=2, wait=True)
    rob.movel_tool((0,0,0,0,0,gripper_direction_set[Go_index]),acc=0.8, vel=2)
    rob.translate_tool((-0.004,0,0),acc=0.1, vel=0.2)
    #rob.translate((0,0,-0.025), acc=0.1, vel=0.4, wait=True)
    rob.translate((0,0,max((ver_pose_Go_stone_set[Go_index][2]-(pos_z-0.32210))*0.7,-0.035)), acc=0.1, vel=0.1, wait=True)
    rob.translate((0,0,max((ver_pose_Go_stone_set[Go_index][2]-(pos_z-0.32210))*0.6,-0.035)), acc=0.001, vel=0.01, wait=False)
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.force.z
    time0 = time.time()
    num_large_force = 0
    while True:
        if num_large_force == 3:
            rob.stopl()
            break
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.force.z<ini_wrench-2:   
            num_large_force += 1
        if time.time()-time0>12:  
            break
    Robotiq.goto(robotiq_client, pos=-3*0.14/48.0+1.12/48.0, speed=0.1, force=0.3, block=False)
    rospy.sleep(1)
    rob.translate((0, 0, 0.08), acc=0.3, vel=0.8)
    go_to_end_place()
    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
    Robotiq.goto(robotiq_client, pos=0.03, speed=0.5, force=10, block=True)
    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)

In [22]:
from shapely.affinity import rotate
from shapely.affinity import translate
from shapely.geometry import Point
from shapely.geometry import LinearRing

def direct_pick_collision_check(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    if gripper_aperture_set[Go_index] > 0.013:
        return False
    finger_profile = [[0.01, gripper_aperture_set[Go_index]/2+0.019], [-0.01, gripper_aperture_set[Go_index]/2+0.019], [-0.01, -gripper_aperture_set[Go_index]/2-0.007], [0.01, -gripper_aperture_set[Go_index]/2-0.007]]
    finger_profile = rotate(LinearRing(finger_profile), -gripper_direction_set[Go_index], origin=Point([0,0]), use_radians=True)
    finger_profile = translate(LinearRing(finger_profile), xoff=ver_pose_Go_stone_set[Go_index][0], yoff=ver_pose_Go_stone_set[Go_index][1], zoff=0.0)
    if finger_profile.within(Point([0.05,0.69]).buffer(0.055))==True:
        return False
    else:
        return True

In [24]:
def distane_to_bowl_center(point_coordinate):
    return sqrt((point_coordinate[0]-0.05)**2+(point_coordinate[1]-0.69)**2)

executed_time_man_array = []
executed_time_vis_array = []
stone_img_count = 0
ver_json_error = False
hori_json_error = False
'''
if 'switching_stone_coordinate_set' in dir():
    while True:
        whether_inherent = raw_input("Whether inherent switching_stone_coordinate_set before? (y or n)")
        if whether_inherent == 'n':
            switching_stone_coordinate_set=[]
            break
        elif whether_inherent == 'y':
            break
else:
    switching_stone_coordinate_set = []
'''
while True:
    #while True:
        #if_continue = raw_input("continue?")
        #if if_continue != '':
            #break
    go_to_safe_place()
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)    
    Robotiq.goto(robotiq_client, pos=2.5*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    go_to_home()
    rospy.sleep(1)
    hori_Go_stone_number = None
    Go_stone_dir_sorted = []
    hori_pose_Go_stone_set = []
    yaw_pitch_normal_set = []
    hori_box_set = []
    hori_box_pixel_set = []
    is_send_stone_img = 1
    time_count_vis_0 = time.time()
    print 'started', stone_img_count
    while True:
        if hori_Go_stone_number != None and hori_Go_stone_number == len(hori_box_pixel_set):
            break
        elif hori_Go_stone_number != None and hori_json_error==True:
            rospy.sleep(0.5)
            break   
        rospy.sleep(0.1)
    vis_time = time.time()-time_count_vis_0
    executed_time_vis_array.append(vis_time)
    print 'vis_time', vis_time
    print hori_pose_Go_stone_set
    print yaw_pitch_normal_set
    time_count_man_0 = time.time()
    priority_list = []
    for index_hori in range(len(hori_pose_Go_stone_set)):
        priority_list.append(('h', index_hori))
    print priority_list
    for index in priority_list:
        if index[0]=='h':
            if hori_pose_Go_stone_set[index[1]]==[None, None, None]:
                continue  
            if hori_pose_Go_stone_set[index[1]][2]<0.02:
                continue
            if hori_pose_Go_stone_set[index[1]][2]>0.075:
                continue
            #if distane_to_bowl_center(hori_pose_Go_stone_set[index[1]])<0.055:
            if True:
                if skim_grasp_execute(index[1])==True:
                    man_time = time.time()-time_count_man_0
                    executed_time_man_array.append(man_time)
                    print 'man_time', man_time
                    break
    else:
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x
        pos_y = rob.get_pose().pos.y
        max_height = np.max(np.array(hori_pose_Go_stone_set)[:,2])
        if max_height>0.045:
            while True:
                if_human_shuffle_finished = raw_input("Human shuffle. Finished? (y or n)")
                if if_human_shuffle_finished == 'y':
                    break
        else:
            if hori_pose_Go_stone_set==[]:
                while True:
                    if_human_shuffle_finished = raw_input("Human shuffle. Finished? (y or n)")
                    if if_human_shuffle_finished == 'y':
                        break 
            else:
                signal.signal(signal.SIGALRM, myHandler)
                signal.setitimer(signal.ITIMER_REAL, 0.005)
                ser.write('1000')
                signal.setitimer(signal.ITIMER_REAL, 0)
                if len(hori_pose_Go_stone_set)<=2:
                    sweep_index_set = range(len(hori_pose_Go_stone_set))
                else:
                    sweep_index_set = random.sample(range(len(hori_pose_Go_stone_set)), 2)
                for hori_index in sweep_index_set:
                    alpha = atan2(0.69-hori_pose_Go_stone_set[hori_index][1], 0.05-hori_pose_Go_stone_set[hori_index][0])
                    if alpha > pi/2 and alpha<=pi:
                        rotate_angle = alpha-3*pi/2
                    else:
                        rotate_angle = alpha+pi/2
                    rob.set_tcp((0.014, 0, 0.3400, 0, 0, 0))
                    rospy.sleep(0.5)
                    rob.movel_tool((0,0,0,0,0,-rotate_angle), acc=0.5, vel=0.8, wait=True) 
                    world2tcp = rob.get_pose()
                    #coordinate_x = max(hori_pose_Go_stone_set[hori_index][0], 0.008)
                    #coordinate_y = min(hori_pose_Go_stone_set[hori_index][1], 0.73)
                    rob.translate((hori_pose_Go_stone_set[hori_index][0]-world2tcp.pos.x-0.002,hori_pose_Go_stone_set[hori_index][1]-world2tcp.pos.y+0.003,0), acc=0.5, vel=0.8, wait=True)
                    rob.movel_tool((0.002,0,0,0,0,0), acc=0.1, vel=0.2, wait=True) 
                    rob.movel_tool((0,0,0,0,20*pi/180,0), acc=0.3, vel=0.8, wait=True) 
                    rob.set_tcp((0, 0, 0, 0, 0, 0))
                    rospy.sleep(0.5)
                    rob.translate((0, 0, -0.065), acc=0.5, vel=0.8, wait=True)
                    rob.translate((0, 0, -0.05), acc=0.001, vel=0.02, wait=False)
                    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
                    ini_wrench = wrench.wrench.force.z
                    torque_memory=[]
                    while True:
                        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
                        torque_memory.append(wrench.wrench.force.z)
                        if len(torque_memory)<9:
                            if wrench.wrench.force.z<ini_wrench-2:
                                rob.stopl()
                                break
                        else:
                            if wrench.wrench.force.z<max(torque_memory[0:9])-2:
                                rob.stopl()
                                break 
                    #rob.translate((0.035*cos(alpha), 0.035*sin(alpha), 0), acc=0.1, vel=0.1, wait=True)
                    point_to_bowl_center_length=sqrt((hori_pose_Go_stone_set[hori_index][0]-0.05)**2+(hori_pose_Go_stone_set[hori_index][1]-0.69)**2)
                    rob.translate((point_to_bowl_center_length*0.8*cos(alpha), point_to_bowl_center_length*0.8*sin(alpha), 0), acc=0.1, vel=0.1, wait=True)
                    #hardtip()
                    rob.translate((0, 0, 0.1), acc=0.3, vel=0.8, wait=True)
                    rob.movej((radians(99.10),radians(-91.67), radians(-107.60), radians(-70.72), radians(89.13), radians(-80.76)), 0.3, 0.7)

started 0
dsag
0
vis_time 4.40692400932
[[0.08579251497025313, 0.6883713801277015, 0.051796046860172196], [0.05828942113554697, 0.6826661073614196, 0.052884471456551396], [0.031396760340885344, 0.6848046605169288, 0.05473887155669399], [0.03554392177474666, 0.6681609375649801, 0.047941289667760234]]
[[1.446441332248135, 0.962107019467485, (-0.13219167471553195, 0.10238448945578467, -0.9508826461892943)], [-1.9049943710603996, 0.7157435896688799, (-0.3195721167275913, 0.07721757890012644, -0.6942969098319626)], [-1.9117752802912458, 0.6640461628266843, (-0.18203453580417836, 0.0173898251375845, -0.4536406467565623)], [-2.2638651554607456, 0.7631795980707291, (0.1143817313437144, 0.004787731544641332, -0.6279457670834072)]]
[('h', 0), ('h', 1), ('h', 2), ('h', 3)]
Go_index ('h', 0)
worldTnormal [-0.1369193063101297, -0.10604611298986528, -0.9848894991212434]
Go_index ('h', 1)
worldTnormal [-0.41599913098787494, -0.10051704775865034, -0.9037925902148225]
[[0.06264674]]
136
man_time 30.533

KeyboardInterrupt: 

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 291, in run
    data = self._get_data()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 338, in _get_data
    tmp = self._s_secondary.recv(1024)
timeout: timed out



In [ ]:
((0.075-0.07441812768708338)+(0.05-0.04897243287305203)+(0.025-0.023395635598574306)+(0.025-0.0254619238759079))/4.0

In [ ]:
((0.675-0.6798453558741636)+(0.675-0.679548104224364)+(0.7-0.7027561247441642)+(0.65-0.6524424649435531))/4.0

In [ ]:
go_to_end_place()

In [ ]:
softtip()

In [23]:
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1180')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)


SerialException: write failed: [Errno 5] Input/output error

[WARN] [1612353292.264282]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


In [ ]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

In [ ]:
from scipy.spatial.transform import Rotation as R
import numpy as np
r = R.from_quat([0, 0, np.sin(np.pi/4), np.cos(np.pi/4)])
r.as_matrix()

In [ ]:
import time
time0 = time.time()
rob.translate((0,0,-0.035), acc=0.001, vel=0.01, wait=True)
print time.time()-time0

In [25]:
go_to_safe_place()

In [23]:
np.mean(executed_time_man_array)

23.93942683393305

In [24]:
np.mean(executed_time_vis_array)

4.260687296207134

In [25]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)

In [82]:
depth_array[127][431]

162.0

In [18]:
    pose_x = 0.05
    pose_y = 0.85
    pose_z = 0.037
    ####################### change tcp pose ############################
    rob.set_tcp((0.0155, 0, 0.343, 0, 0, 0))  
    rospy.sleep(0.5)
    ####################################################################
    go_to_safe_place()    
    worldPstone = np.array([pose_x, pose_y, pose_z, 1])
    #print 'worldPstone', worldPstone
    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))
    rob.movel_tool((tcpTstone[0], tcpTstone[1]-0.001, 0, 0, 0, 0), acc=0.1, vel=0.3, wait=True)

In [97]:
go_to_home()